# Workshop 1
Hi and welcome to the first of a series of workshops on using MODFLOW-6 with Flopy.

## This workshop agenda
1. Introductions and outline (10 minutes)
2. Setup, environment configuration and downloads (10 minutes, wishful)
3. Flopy Github and MF6 Github (10 minutes)
4. Simulations and models (10 minutes)
5. Setting up your first simulation - the mfsim.nam file (10 minutes)
6. Setting up a model - the model.nam file (10 minutes)
7. Temporal discretisation - the tdis package (20 minutes)
8. Spatial discretisation - DIS, DISV and DISU packages (40 minutes)

    a.2D (Dupuit-Forcheimer)

    b.2D -Xsection

    c.3D


# Imports
These are someof the most common libraries that i use when working on building a model with flopy. You will notice that some libraries are imported with an alias. Hopefully you are all familiar with the reasons for this. If not don't worry it will become clearer later. I find it useful to always check which version of the library you are using in case you get any issues with errors that may be specific to the version of the library that you are using.

In [1]:
import os
import sys
import shutil
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import flopy
import pyemu

print(f"Pandas version = {pd.__version__}")
print(f"Numpy version = {np.__version__}")
print(f"GeoPandas version = {gpd.__version__}")
print(f"Flopy version = {flopy.__version__}")
print(f"Pyemu version = {pyemu.__version__}")

Pandas version = 2.0.3
Numpy version = 1.25.2
GeoPandas version = 0.13.2
Flopy version = 3.4.1
Pyemu version = 1.3.2


# Setting up some folders to dump things in later
We will be creating files and perhaps using files from other utilities so it is nice to have things organised. You can do this however you want but as a simple example you can follow what I do below. Lets first take a look at where we are on the computer by using the os library.

In [2]:
os.getcwd()

'c:\\Working\\NCGRT_IntroFlopy'

The path above is where we are working right now. Lets make a new folder specific to this workshop.

In [3]:
ws1 = os.path.join('workshop_1') # here we are making a path not creating the folder
if os.path.exists(ws1): # here we are asking if the path exists on the computer. 
    pass # if it does exist, do nothing
else:
    os.mkdir(ws1) # if it doesn't exist then make the folder

Lets make a few extra paths to folders that we will use later. Notice how the workshop_1 folder path (which is ws1) is now the first entry in the join chain. This means that the paths we are making will be sub-directories of the workshop_1 folder.

In [4]:
gis_f = os.path.join(ws1,'GIS') # creating a sub-directory path for our gis input/output
model_f = os.path.join(ws1,'model') # creating a sub-directory path for our model input/output
plots_f = os.path.join(ws1,'plots') # creating a sub-directory path for our plots

Now we will make a few sub-directories inside our workshop folder by using a for loop from a list of strings called subdirectory_list. Once you execute the code below check you folder for the subdirectories.

In [5]:
subdirectory_pathlist = [gis_f,model_f,plots_f] # this is our list of sub-directory paths, each one is a path string
for path in subdirectory_pathlist: # for each path in the list above
    if os.path.exists(path): # check if the folder already exists
        pass # if it does exist then do nothing
    else:
        os.mkdir(path) # otherwise make the folder

# Setting up a MF6 simulation
The simulation namefiel is required for all MF6 modelling work. What you are creating next is the mfsim.nam file. This is the file at the highest level when it comes to all modelling work in you specfic project using mf6. The mfsim.nam file is what the MF6 executable will look for as soon as you try to run a model in you project folder.

In [6]:
sim_name = "MySim" # give your simulation a name, usually I use a project name but this is entirley up to you
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f) # this one is mandatory if you want to make sure that the model files are created in the folder we want

Lets see if we can write the mfsim.nam file using Flopy's package write function.

In [7]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


AttributeError: 'NoneType' object has no attribute 'write'

Hmm, looks like it failed trying to find the tdis package object but it did succeed in writing the mfsim.nam file before failing. The write_simualtion() method expects a certain number of package objects to be present or it will fail. We'll discuss this more later but for now let's take a quick peek at the mfsim.nam file. You can execute the code below to view the contents of the file or alternativley open the file in its folder with your favourite text editor. Do you know where to look for the file?

In [8]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"mfsim.nam"))]

# File generated by Flopy version 3.4.1 on 09/15/2023 at 20:20:41.
BEGIN options
END options

BEGIN exchanges
END exchanges



Open the mf6io.pdf document to see what else could be in this file if we did things a bit differently.
We will discuss what the other sections are in the document and whether you add them explicitly to the sim object of if Flopy will handle that for you.

Things to think about.

Can I remake the object? Can a simulation have more than one simulation object? Are there any conflicts with options? What happens if enter something incorrectly? Try to remake the sim object with all options turned on.

In [16]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f, # this one is mandatory if you want to make sure that the model files are created in the folder we want
                             continue_=True,
                             nocheck=True,
                             lazy_io=True,
                             memory_print_option="ALL",
                             write_headers=False) 

In [17]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


AttributeError: 'NoneType' object has no attribute 'write'

In [18]:
_ = [print(line.rstrip()) for line in open(os.path.join(model_f,"mfsim.nam"))]

BEGIN options
  CONTINUE
  NOCHECK
  MEMORY_PRINT_OPTION  all
END options

BEGIN exchanges
END exchanges



Wait a minute what about maxerrors?

In [44]:
sim.package_filename_dict['mfsim.nam'].blocks['options'].datasets_keyword

{('continue',): <flopy.mf6.data.mfstructure.MFDataStructure at 0x1d037e80040>,
 ('nocheck',): <flopy.mf6.data.mfstructure.MFDataStructure at 0x1d037e0f850>,
 ('memory_print_option',): <flopy.mf6.data.mfstructure.MFDataStructure at 0x1d037e0ffa0>,
 ('maxerrors',): <flopy.mf6.data.mfstructure.MFDataStructure at 0x1d037ca35b0>,
 ('print_input',): <flopy.mf6.data.mfstructure.MFDataStructure at 0x1d037d15060>}

Hmm, it looks like it should be there.

In [49]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, 
                             exe_name="mf6", # this is a path to your exe for mf6, if it is available system wide then all you need is "mf6"
                             verbosity_level=1, # this is optional but can be handy
                             sim_ws=model_f, # this one is mandatory if you want to make sure that the model files are created in the folder we want
                             continue_=True,
                             nocheck=False,
                             lazy_io=True,
                             memory_print_option="ALL",
                             write_headers=False,
                             maxerrors=20) 

TypeError: MFSimulation.__init__() got an unexpected keyword argument 'maxerrors'

Out of options. Time to go to GitHub or dig into the source code.

In [7]:
from helpers import ws1_mod
ws1_mod(sim)

()

In [8]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model MyModel...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package chd_0...
INFORMATION: maxbound in ('gwf6', 'chd', 'dimensions') changed to 2 based on size of stress_period_data
    writing package oc...


sim_name = MySim
sim_path = c:\Working\NCGRT_workshops\workshop_1\model
exe_name = mf6

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = MySim


###################
Package MySim.tdis
###################

package_name = MySim.tdis
filename = MySim.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = MySim


###################
Package ims_-1
###################

package_name = ims_-1
filename = MySim.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = MySim


@@@@@@@@@@@@@@@@@@@@
Model MyModel
@@@@@@@@@@@@@@@@@@@@

name = MyModel
model_type = gwf6
version = mf6
model_relative_path = .

###################
Package dis
###################

package_name = dis
filename = MyModel.dis
package_type = dis
model_or_simulation_package = model
model_name = MyModel


###################
Package ic
########